In [1]:
import json
import seaborn as sns
import requests
import time 
import gzip
import matplotlib.pyplot as plt
import sys 
sys.path.append("/scratch/izar/kapps/DEX-Cyclic-Arbitrage/")
from helper import *
from config.get import cfg
import pandas as pd
import numpy as np

print("Loading filtered cycles...")
filtered_cycles = load_cycles("filtered")

Loading filtered cycles...


In [2]:
cycle_ids = []
revenues = []
costs = []
token1 = []
token2 = []
token3 = []
for line in filtered_cycles:
    line = json.loads(line)
    cycle_ids.append(line["cycle_id"])
    revenues.append(line["revenue"])
    costs.append(line["cost"])
    token1.append(line["path"][0])
    token2.append(line["path"][2])
    token3.append(line["path"][4])

# Build features

In [5]:
features = pd.DataFrame({
    "cycle_id":cycle_ids,
    "revenues":revenues,
    "costs":costs,
    "token1":token1,
    "token2":token2,
    "token3":token3
    })
features = features.astype({'cycle_id': 'int32',"revenues":np.float64,"costs":np.float64})

In [6]:
features["profits"] = features["revenues"]-features["costs"] 
features["profitability"] = features["profits"]>0

In [8]:
features.to_csv(cfg["files"]["features"])

## Train/Test features

In [10]:
train_ids = np.load(cfg['files']['train_ids']).astype(int)
test_ids  = np.load(cfg['files']['test_ids']).astype(int)
train_ids = pd.DataFrame({"cycle_id":train_ids})
test_ids  = pd.DataFrame({"cycle_id":test_ids})

features_i = features.set_index('cycle_id')

f_train = train_ids.join(features_i,on="cycle_id",lsuffix="_")
f_test  = test_ids.join(features_i,on="cycle_id",lsuffix="_")

f_train

,cycle_id,revenues,costs,token1,token2,token3,profits,profitability
0,51068,1.488251e+17,4.598268e+16,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xbf2179859fc6D5BEE9Bf9158632Dc51678a4100e,0xC28E27870558cF22ADD83540d2126da2e4b464c2,1.028424e+17,True
1,46217,5.474499e+16,2.786625e+16,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0x9676EE2eDCc830baf858dad8E56ae1d251783Eb3,0xdAC17F958D2ee523a2206206994597C13D831ec7,2.687874e+16,True
2,44104,8.136130e+16,5.425344e+16,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xf29e46887FFAE92f1ff87DfE39713875Da541373,0x9fBFed658919A896B5Dc7b00456Ce22D780f9B65,2.710785e+16,True
3,50240,3.060971e+17,1.233920e+17,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xC28E27870558cF22ADD83540d2126da2e4b464c2,0xbf2179859fc6D5BEE9Bf9158632Dc51678a4100e,1.827051e+17,True
4,44997,1.484209e+17,8.918950e+16,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xa93D5Cfaa41193b13321c035b4bDD2B534172762,0x6B3595068778DD592e39A122f4f5a5cF09C90fE2,5.923137e+16,True
...,...,...,...,...,...,...,...,...
9328,43218,1.555328e+16,2.399796e+16,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xdAC17F958D2ee523a2206206994597C13D831ec7,0xa93D5Cfaa41193b13321c035b4bDD2B534172762,-8.444685e+15,False
9329,50252,9.261110e+17,7.046000e+16,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xbf2179859fc6D5BEE9Bf9158632Dc51678a4100e,0xC28E27870558cF22ADD83540d2126da2e4b464c2,8.556510e+17,True
9330,2346,2.322861e+16,1.083954e+16,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xF80D589b3Dbe130c270a69F1a69D050f268786Df,0x469eDA64aEd3A3Ad6f868c44564291aA415cB1d9,1.238907e+16,True
9331,49646,1.518578e+17,5.859790e+16,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,0xf3A2ace8e48751c965eA0A1D064303AcA53842b9,0x2b591e99afE9f32eAA6214f7B7629768c40Eeb39,9.325991e+16,True


In [11]:
f_train.to_csv(cfg["files"]["features_train"])
f_test.to_csv(cfg["files"]["features_test"])

In [13]:
a=2